# NLP Tweet Analyzer Kaggle Submission

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

# NLP Preprocessing
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# NLP Viz
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer

# Data Modelling
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, recall_score
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier
xgboost.config_context(verbosity=0) # Silect XGBoost

# Monitoring progress
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

nltk.download('stopwords')

config = {
    'test_size': 0.2,
    'CV_splits': 5,
    'seed': 14,
    'n_cores': 16,
    'max_features': 10000
}

/Users/miguelcachosoblechero/opt/anaconda3/envs/Kaggle/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguelcachosoblechero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# 1- Preprocessing and Feature Engineering
# Load dataset
input_path = "../input"
raw_tweets_train = pd.read_csv(os.path.join(input_path, "train.csv")).drop(['id'], axis=1)
raw_tweets_test = pd.read_csv(os.path.join(input_path, "test.csv"))

# Extract data and labels
X_train = raw_tweets_train.drop(['target'], axis=1)
y_train = raw_tweets_train.target.values
X_test = raw_tweets_test

# Tokenize + Stop Words + BoW
CountVec = CountVectorizer(stop_words='english', max_features=config['max_features'])
X_train_bow = CountVec.fit_transform(X_train.text)
X_test_bow = CountVec.transform(X_test.text)

In [14]:
raw_tweets_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [16]:
# 2- Data modelling
# Select your model
target_model = MultinomialNB()

# Train your model
target_model.fit(X_train_bow, y_train)

# Generate predictions
results = target_model.predict(X_test_bow)

# Store results
pd.DataFrame({"id": raw_tweets_test.id,
              "target": results}).set_index("id").to_csv("../submission/nlp_submission.csv")